### Item 50: Use Packages to Organize Modules and Provide Stable APIs

* As the size of a program's codebase grows, it's natural to reorganize its structure.
* Split larger functions into smaller functions, refactor data structures into helper classes.
    * See `Item 22`: Prefer Helper Classes Over Bookkeeping with Dictionaries and Tuples.
    * Separate functionality into various modules that depend on each other.
* Python provides packages.
    * `Packages` are modules that contain other `modules`.
    * In most cases, packages are defined by putting an empty file named `__init__`.py into a directory.
    * Once `__init__.py` is present, any other Python files in that directory will be available for import using a path relative to the directory.

* To import the `utils` module, you use the absolute module name that includes the package directory's name

In [ ]:
# # main.py
# from mypackage import utils

* Note
    * Python 3.4 introduces `namespace` packages, a more flexible way to define packages.
    * `Namespace` packages can be composed of modules from completely separate directories, zip archives, or even remote systems.
    * See `PEP 420` (http://www.python.org/dev/peps/pep-0420/).

* The functionality provided by packages has two primary purposes in Python programs.

#### Namespaces

* The first use o fpackages is to help divide your modules into separate namespaces.
    * This allows you to have many modules with the same filename but different absolute paths that are unique.
    * e.g.
        * A program that imports attributes from two modules with the same name.
        * Best try to avoid `imported name conflict`!

In [ ]:
# # main.py
# from analysis.utils import log_base2_bucket
# from frontend.utils import stringify

# bucket = stringify(log_base2_bucket(33))

* Problem 1

    * This approach breaks down when the functions, classes, or submoules defined in packages have the same names.
    * Importing the attributes directly won't work because the second import statement will overwrite the value of inspect in the current scope.

In [ ]:
# # main2.py
# from analysis.utils import inspect
# from frontend.util import inspect  # overwrites!

* Solution:

    * Use the `as` clause of the import statement to rename whatever you've imported for the current scope.
    * The `as` clause can be used to rename anything you retrieve with the `import` statement, including entire modules.

In [ ]:
# # main3.py
# from analysis.utils import inspect as analysis_inspect
# from frontend.utils import inspect as frontend_inspect

# value = 33
# if analysis_inspect(value) == frontend_inspect(value):
#     print('Inspection equal!')

* Note

    * Another approach for avoiding `imported name conflicts` is to always access names by their highest unique module name.
    * e.g.
        * First import:
            * import analysis.utils
            * import frontend.utils
        * Then, access the inspect functions with the full paths:
            * analysis.utils.inspect
            * frontend.utils.inspect

#### Stable APIs

* The second use of packages in Python is to provide strict, stable APIs for external consumers.
* When writing an API for wider consumption, like an open source package, you'll want to provide stable functionality that doesn't change between releases.
    * See `Item 48`: Know Where to Find Community-Built Modules.
* To ensure that happens, it's important to hide your internal code organization from external users.
    * This enables you to refactore and improve your package's internal modules without breaking existing users.

* Python can limit the surface area exposed to API consumers by using the `__all__` special attribute of a module or package.
* The value of `__all__` is a list of every name to export from the module as part of its `public API`.
    * When you run `from foo import *`, only the attributes in `foo.__all__` will be imported from foo.
    * If `__all__` isn't present in foo, then only public attributes (those without a leading underscore) are imported.
        * See `Item 27`: Prefer Public Attributes Over Private Ones

In [ ]:
# # models.py
# __all__ = ['Projectile']

# class Projectile(object):
#     def __init__(self, mass, velocity):
#         self.mass = mass
#         self.velocity = velocity
        
# # utils.py
# from . models import Projectile

# __all__ = ['simulate_collision']

# def _dot_product(a, b):
#     ...

# def simulate_collision(a, b):
#     ...

* All of the public parts of this API as a set of attributes that are available on the mypackage module.
* Consumers can always import directly from mypackage instead of importing `from mypackage.models` or `mypackage.utils`.
* This ensures that the API consumer's code will continue to work even if the internal organization of mypackage changes:
    * e.g. models.py is deleted

* To do this with Python packages, you need to modify the `__init__.py` file in the mypackage directory.
* This file actually becomes the contents of the mypackage module when it's imported.
    * You can specify an explicit API for mypackage by limiting what you import into `__init__.py`.
    * Since all of the internal modules already specify `__all__`, I can expose the public interface of mypackage by simply importing everything from the internal modules and updating `__all__` accordingly.

In [ ]:
# # __init__.py
# __all__ = []

# from . models import *
# __all__+= models.__all__

# from . utils import *
# __all__ += utils.__all__

* Consumer of the API that directly imports from mypackage instead of accessing the inner modules

In [ ]:
# # api_consumer.py
# from mypackage import *

# a = Projectile(1.5, 3)
# b = Projectile(4, 1.7)
# after_a, after_b = simulate_collision(a, b)

* Internal-only functions like `mypackage.utils._dot_product` will not be available to the API consumer on mypackage.
    * They weren't present in `__all__`.
    * Being omitted from `__all__` means they weren't imported by the `from mypackage import *` statement.
    * The internal-only names are effectively hidden.

* Note:
    
    * This whole approach works great when it's important to provide an explicit, stable API.
    * If you're building an API for use between your own modules, the functionality of `__all__` is probably unnecessaryand should be avoided.
        * The `namespacing` provided by packages is usually enough for a team of programmers to collaborate on large amounts of code.
        * They control while maintaining reasonable interface boundaries.

#### Beware of import `*`

* Import statements like `from x import y` are clear because the source of y is explicitly the x package or module.
* Wildcard imports like `from foo import *` make code more difficult to understand.
    * `from foo import *` hides the source of names from new readers of the code.
        * If a module has multiple `import *` statements, you'll need to check all of the referenced modules to figure out where a name was defined.
    * Names from `import *` statements will overwrite any conflicting names within the containing module.
        * This can lead to strange bugs caused by accidental interactions between your code and overlapping names from multiple `import *` statements.
    
        
    [Important!]
* Avoid `import *` in your code and explicitly import names with the `from x import y` style.

### Things to Remember

* Packages in Python are modules that contain other modules.
    * Packages allow you to organize your code into separate, non-conflicting `namespaces` with unique absolute module names.
* Simple packages are defined by adding an `__init__.py` file to a directory that contains other source files.
    * These files become the child modules of the directory's package.
    * Package directories may also contain other packages.
* You can provide an explicit API for a module by listing its publicly visibe names in its `__all__` special attribute.
* You can hide a package's internal implementation:
     * By only importing public names in the package's `__init__.py` file.
     * By naming internal-only members with a leading underscore.
* When collaborating within a single team or on a single codebase, using `__all__` for explicit APIs is probably unnecessary and should be avoided.